# A/B-тестирование

<div class="alert alert-info">
<h2> Привет!<a class="tocSkip"></h2>
<hr>  
Я - Сергей. Рад что ты читаешь это текст :) Сделаю пару ремарок общего характера. В описании выполняемых действий, комментариях, я использую местоимение 'мы', так как считаю что ты - ревьювер, проверя код становиться моим соучастником ('соучавствующие программирование') и мы вместе проходим по пути исследования. Да и сформировлась привычка так писать в ходе написания научных работ.   
NB! - так помечаю важные на мой взгдял идеи, которые стоит взять на заметку; инсайты, проработка которых требует большего знания предметной области.   
Надеюсь мои многословные комментарии тебя не утомят.    
    
</div>
    
<br/>

**Содержание**    
[1. Описание проекта](#description)    
[2. Загрузка и предобработка данных](#review)    
[3. Исследовательский анализ данных (EDA)](#EDA)   
[3.1. Исследование конверсии в воронке на разных этапах](#conversion)    
[3.2. Анализ распределения количества событий на пользователя в выборках](#distribution)    
[3.3. Присутствие в выборках одни и те же пользователи, пересечение с аудиториями других тестов](#double)    
[3.4. Распределение событий по дням и пересчение с другими маркетинговыми активностями](#cross)    
[3.5. Равномерность распределения по тестовым группам и правильность их формирования](#equability)   
[4. Оценка результатов А/В-теста](#AB_test)   
[4.1. Анализ результата A/B-теста](#AB_result)  
[4.2. Проверка статистической разницы долей z-критерием](#AB_z)   
[5. Выводы и рекомендации](#summary)   

## Описание проекта<a id="description"></a>     
      
### Постановка задачи

Ваша задача — провести оценку результатов A/B-теста. В вашем распоряжении есть датасет с действиями пользователей, техническое задание и несколько вспомогательных датасетов.   

- Оцените корректность проведения теста   
- Проанализируйте результаты теста   

Чтобы оценить корректность проведения теста, проверьте:   

- пересечение тестовой аудитории с конкурирующим тестом,   
- совпадение теста и маркетинговых событий, другие проблемы временных границ теста.   

### Техническое задание   

- Название теста: `recommender_system_test`;   
- группы: А — контрольная, B — новая платёжная воронка;   
- дата запуска: 2020-12-07;   
- дата остановки набора новых пользователей: 2020-12-21;   
- дата остановки: 2021-01-04;   
- аудитория: 15% новых пользователей из региона EU;   
- назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;   
- ожидаемое количество участников теста: 6000.   
- ожидаемый эффект: за 14 дней с момента регистрации пользователи покажут улучшение каждой метрики не менее, чем на 10%:   
    - конверсии в просмотр карточек товаров — событие `product_page`,   
    - просмотры корзины — `product_cart`,   
    - покупки — `purchase`.   
   
### Описание данных

`ab_project_marketing_events.csv` — календарь маркетинговых событий на 2020 год.

Структура файла:

- `name` — название маркетингового события;
- `regions` — регионы, в которых будет проводиться рекламная кампания;
- `start_dt` — дата начала кампании;
- `finish_dt` — дата завершения кампании.

`final_ab_new_users.csv` — пользователи, зарегистрировавшиеся с 7 по 21 декабря 2020 года.

Структура файла:

- `user_id` — идентификатор пользователя;
- `first_date` — дата регистрации;
- `region` — регион пользователя;
- `device` — устройство, с которого происходила регистрация.

`final_ab_events.csv` — действия новых пользователей в период с 7 декабря 2020 по 4 января 2021 года.

Структура файла:

- `user_id` — идентификатор пользователя;
- `event_dt` — дата и время покупки;
- `event_name` — тип события;
- `details` — дополнительные данные о событии. Например, для покупок, `purchase,` в этом поле хранится стоимость покупки в долларах.

`final_ab_participants.csv` — таблица участников тестов.

Структура файла:

- `user_id` — идентификатор пользователя;
- `ab_test` — название теста;
- `group` — группа пользователя.   
   
### В рамках проекта мы:   
1. Загрузим и предобработаем данные    
1. Проведём исследовательский анализ данных (EDA), а именно:   
* Исследование конверсии в воронке на разных этапах   
* Анализ распределения количества событий на пользователя в выборках   
* Проверку присутствие в выборках одни и те же пользователи, пересечение с аудиториями других тестов   
* Посмотрим распределение событий по дням и пересечение с другими маркетинговыми активностями   
* Посмотрим равномерность распределения по тестовым группам и правильность их формирования   
3. Оценим результатов А/В-теста - проведём:   
* Анализ результата A/B-теста   
* Проверку статистической разницы долей z-критерием   
4. По итогам работы сформулируем выводы    
   

---

## Загрузка и предобработка данных<a id="review"></a>      
На данном этапе мы:   
* Загрузим данные   
* Проверим датафремы на наличие пропусков и дубликатов, несоответствие типов данных - и при необходимости выполним обработку данных
* Посмотрим описательную статистку на предмет выбросов   

**Загрузка и исследование данных**

In [1]:
# Импортруем необходимые библиотеки
import pandas as pd
import numpy as np
import plotly.express as px
from plotly import graph_objects as go
import matplotlib.pyplot as plt
import math as mth
from scipy import stats as st
from io import BytesIO
import requests

In [2]:
#Загрузим данные с гуглдиска
spreadsheet_ab_project_marketing_events = '1XI4XBK_-KWXo_tOPOff1RNwq4zaNuisLCNcHVBRlaHk'
file_name_ab_project_marketing_events = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_ab_project_marketing_events)
r_ab_project_marketing_events = requests.get(file_name_ab_project_marketing_events)
ab_project_marketing_events = pd.read_csv(BytesIO(r_ab_project_marketing_events.content)) # Датасет с источниками рекламного трафика

In [3]:
spreadsheet_final_ab_events = '13zdgsxA9-lz5DhGCaVGvLm-qtc-SpEdGjxu0btJhR2s'
file_name_final_ab_events = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_final_ab_events)
r_final_ab_events = requests.get(file_name_final_ab_events)
final_ab_events = pd.read_csv(BytesIO(r_final_ab_events.content)) # Датасет с активностями игроков

In [4]:
spreadsheet_final_ab_new_users = '1FpTyIJI44F_dOE-tlswsoDQyANFxQwpSRSZE3tnycWE'
file_name_final_ab_new_users = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_final_ab_new_users)
r_final_ab_new_users = requests.get(file_name_final_ab_new_users)
final_ab_new_users = pd.read_csv(BytesIO(r_final_ab_new_users.content)) # Датасет с источниками откуда пришли пользователи

In [5]:
spreadsheet_final_ab_participants = '1Koew7yM5fPkP64uLr-BKDLuhcPzmcE8Ld5HWM4z4z9o'
file_name_final_ab_participants = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_final_ab_participants)
r_final_ab_participants = requests.get(file_name_final_ab_participants)
final_ab_participants = pd.read_csv(BytesIO(r_final_ab_participants.content)) # Датасет с источниками откуда пришли пользователи

In [6]:
# Посмотрим на датафрейм ab_project_marketing_events
display(ab_project_marketing_events.head()) # Выведем 5 верхних записей
print('-------------------------------------')
print('Описание датасета')
ab_project_marketing_events.info() # Посмотрим описание датафрейма
print('-------------------------------------')
print('Пропуски данных')
ab_project_marketing_events.isna().sum() # Проверим пропуски

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11


-------------------------------------
Описание датасета
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes
-------------------------------------
Пропуски данных


name         0
regions      0
start_dt     0
finish_dt    0
dtype: int64

In [7]:
# Проверим дублирующиеся записи
print('Дубликаты')
ab_project_marketing_events.duplicated().sum()

Дубликаты


0

In [8]:
print('Описательная статистика')
ab_project_marketing_events.describe()

Описательная статистика


,name,regions,start_dt,finish_dt
count,14,14,14,14
unique,14,6,14,14
top,Christmas&New Year Promo,APAC,2020-12-25,2021-01-03
freq,1,4,1,1


В датафреме ab_project_marketing_events 14 записей (строк) и 4 колонки. Имена колонок соответствуют хорошему стилю написания. Пропусков данных и дубликатов - нет. Тип данных колонок, хранящих дату, не соответвует её содержимому - приведём их к типу date.

In [9]:
# Посмотрим на датафрейм final_ab_events
display(final_ab_events.head()) # Выведем 5 верхних записей
print('-------------------------------------')
print('Описание датасета')
final_ab_events.info() # Посмотрим описание датафрейма
print('-------------------------------------')
print('Пропуски данных')
final_ab_events.isna().sum() # Проверим пропуски

,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 9:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 4:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99


-------------------------------------
Описание датасета
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     440317 non-null  object 
 1   event_dt    440317 non-null  object 
 2   event_name  440317 non-null  object 
 3   details     62740 non-null   float64
dtypes: float64(1), object(3)
memory usage: 13.4+ MB
-------------------------------------
Пропуски данных


user_id            0
event_dt           0
event_name         0
details       377577
dtype: int64

In [10]:
# Проверим дублирующиеся записи
print('Дубликаты')
final_ab_events.duplicated().sum()

Дубликаты


0

In [11]:
print('Описательная статистика')
final_ab_events.describe()

Описательная статистика


,details
count,62740.000000
mean,23.877631
std,72.180465
min,4.990000
25%,4.990000
50%,4.990000
75%,9.990000
max,499.990000


В датафреме final_ab_events 440317 записей (строк) и 4 колонки. Имена колонок соответствуют хорошему стилю написания. Есть пропуски данных в колонке details. Дубликатов - нет. Тип данных колоноки event_dt не соответствуют содержимому, привдём его типу data. Максимальное значение в колонке  details - 499.99, возможно это выброс.

In [12]:
# Посмотрим на датафрейм final_ab_new_users
display(final_ab_new_users.head()) # Выведем 5 верхних записей
print('-------------------------------------')
print('Описание датасета')
final_ab_new_users.info() # Посмотрим описание датафрейма
print('-------------------------------------')
print('Пропуски данных')
final_ab_new_users.isna().sum() # Проверим пропуски

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone


-------------------------------------
Описание датасета
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     61733 non-null  object
 1   first_date  61733 non-null  object
 2   region      61733 non-null  object
 3   device      61733 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB
-------------------------------------
Пропуски данных


user_id       0
first_date    0
region        0
device        0
dtype: int64

In [13]:
# Проверим дублирующиеся записи
print('Дубликаты')
final_ab_new_users.duplicated().sum()

Дубликаты


0

In [14]:
print('Описательная статистика')
final_ab_new_users.describe()

Описательная статистика


,user_id,first_date,region,device
count,61733,61733,61733,61733
unique,61733,17,4,4
top,D72A72121175D8BE,2020-12-21,EU,Android
freq,1,6290,46270,27520


В датафреме final_ab_new_users 61733 записей (строк) и 4 колонки. Имена колонок соответствуют хорошему стилю написания. пропусков и дубликатов - нет. Тип данных колоноки first_date не соответствуют содержимому, приведём её к типу date.

In [15]:
# Посмотрим на датафрейм final_ab_participants
display(final_ab_participants.head()) # Выведем 5 верхних записей
print('-------------------------------------')
print('Описание датасета')
final_ab_participants.info() # Посмотрим описание датафрейма
print('-------------------------------------')
print('Пропуски данных')
final_ab_participants.isna().sum() # Проверим пропуски

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test


-------------------------------------
Описание датасета
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18268 entries, 0 to 18267
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  18268 non-null  object
 1   group    18268 non-null  object
 2   ab_test  18268 non-null  object
dtypes: object(3)
memory usage: 428.3+ KB
-------------------------------------
Пропуски данных


user_id    0
group      0
ab_test    0
dtype: int64

In [16]:
# Проверим дублирующиеся записи
print('Дубликаты')
final_ab_participants.duplicated().sum()

Дубликаты


0

In [17]:
print('Описательная статистика')
final_ab_participants.describe()

Описательная статистика


,user_id,group,ab_test
count,18268,18268,18268
unique,16666,2,2
top,0FDFDA0B2DEC2D91,A,interface_eu_test
freq,2,9655,11567


В датафреме final_ab_participants 18268 записей (строк) и 3 колонки. Имена колонок соответствуют хорошему стилю написания. пропусков и дубликатов - нет. Типы данных колонокок соответствуют содержимому.

**Предобработка данных**

In [18]:
# Обработаем пропуски
# Проверим, все ли заполненые данные имеют отношение к purchase
final_ab_events[final_ab_events['details'].notna()]['event_name'].unique()

array(['purchase'], dtype=object)

In [19]:
# Проверим какие события происходили
final_ab_events['event_name'].unique()

array(['purchase', 'product_cart', 'product_page', 'login'], dtype=object)

In [20]:
# Т.к.пропуски не относятся к событию purchase они не содержат данных о стоимости покупок  - 
# заполним пропущенные значений нулями
final_ab_events['details'] = final_ab_events['details'].fillna(0)
#final_ab_events.info()

In [21]:
# Изменение типов данных на data
ab_project_marketing_events['start_dt'] = pd.to_datetime(ab_project_marketing_events['start_dt'])
ab_project_marketing_events['finish_dt'] = pd.to_datetime(ab_project_marketing_events['finish_dt'])
final_ab_new_users['first_date'] = pd.to_datetime(final_ab_new_users['first_date'])
final_ab_events['event_dt'] = pd.to_datetime(final_ab_events['event_dt'])
#ab_project_marketing_events.info()
#final_ab_new_users.info()
#final_ab_events.info()

**Вывод**  
Мы загрузи и исследоваили данные, обнаруженные пропуски данных и несоответствие типов данных колонок их содержимому - исправили.

---

## Исследовательский анализ данных (EDA)<a id="EDA"></a>   
На данном этапе мы:   
* будем исследовать конверсию в воронке на разных этапах   
* посмотрим обладают ли выборки одинаковыми распределениями количества событий на пользователя   
* посмотрим присутствуют ли в выборках одни и те же пользователи и не пересекаются аудитория теста с аудиториями других тестов      
* посмотрим как число событий распределено по дням и не перескался ли тест с другими маркетинговыми активностями    
* проверим равномерность распределения по тестовым группам и правильность их формирования 

### Исследование конверсии в воронке на разных этапах<a id="conversion"></a>

In [22]:
# Объеденим дествия пользователей и  пользователей попавших в тест recommender_system_test
events_groups = final_ab_events.merge(final_ab_participants[final_ab_participants['ab_test'] == 'recommender_system_test'],
                      how='inner',
                      on='user_id')
display(events_groups.head())

,user_id,event_dt,event_name,details,group,ab_test
0,831887FE7F2D6CBA,2020-12-07 06:50:29,purchase,4.99,A,recommender_system_test
1,831887FE7F2D6CBA,2020-12-09 02:19:17,purchase,99.99,A,recommender_system_test
2,831887FE7F2D6CBA,2020-12-07 06:50:30,product_cart,0.00,A,recommender_system_test
3,831887FE7F2D6CBA,2020-12-08 10:52:27,product_cart,0.00,A,recommender_system_test
4,831887FE7F2D6CBA,2020-12-09 02:19:17,product_cart,0.00,A,recommender_system_test


In [23]:
# Сгруппируем уникальных пользователей по событиям и подсичтаем число пользовтелей для каждого
# Для этого напишем функцию
def funnel_events(data, title):
    events = (data
                        .groupby('event_name')
                        .agg({'user_id':'nunique'})
                        .sort_values(by = 'user_id', ascending=False)
                        .reset_index())
    # Построем воронку
    fig = go.Figure(go.Funnel(
        y = events['event_name'],
        x = events['user_id'],
        textinfo = "value+percent previous"
        ))

    fig.update_layout(title_text=title)
    fig.show()

In [24]:
funnel_events(final_ab_events, 'Конверсия пользователей')

In [25]:
funnel_events(events_groups[events_groups['group'] == 'A'], 'Конверсия пользователей в группе A')

In [26]:
funnel_events(events_groups[events_groups['group'] == 'B'], 'Конверсия пользователей в группе B')

В обеих группах больше половины пользователей зарегистрировавшихся на сайте, перешли на страницу с товаром. Почти половина пользователей со страницы с товаром перешли к оплате товаров.   
**NB!** Количество оплаченых товаров превышает количество товаров добавленых в корзину - скорей всего у поользователей есть возможность перейти к оплате товара сразу со страницы товара.

**Найдём пользователей совершившвых все шаги воронки без пропусков:**   
1. Авторизация  
1. Страница товара   
1. Коризина   
1. Оплата   

In [27]:
# Для этого также используем функцию 
def user_events_by_steps(data, title):
    users = data.pivot_table(
        index='user_id', 
        columns='event_name', 
        values='event_dt',
        aggfunc='min')

    step_1 = ~users['login'].isna() # впишите ваш код сюда
    step_2 = step_1 & (users['product_page'] > users['login']) # впишите ваш код сюда
    step_3 = step_2 & (users['product_cart'] > users['product_page']) # впишите ваш код сюда
    step_4 = step_3 & (users['purchase'] > users['product_cart']) # впишите ваш код сюда

    n_login = users[step_1].shape[0]
    n_product_page = users[step_2].shape[0]
    n_product_cart = users[step_3].shape[0]
    n_purchase = users[step_4].shape[0]

    print('Авторизация:', n_login)
    print('Страница товара:', n_product_page)
    print('Корзина:', n_product_cart)
    print('Оплата:', n_purchase)
    
    # построем воронку
    fig = go.Figure(go.Funnel(
        y = ['login', 'product_page', 'product_cart', 'purchase'],
        x = [n_login, n_product_page, n_product_cart, n_purchase],
        textinfo = "value+percent previous"
        ))

    fig.update_layout(title_text=title)
    fig.show()
user_events_by_steps(final_ab_events, 'Конверсия пользователей (последовательно)')

Авторизация: 58697
Страница товара: 22119
Корзина: 1347
Оплата: 6


In [28]:
user_events_by_steps(events_groups[events_groups['group'] == 'A'], 'Конверсия пользователей (последовательно) в группе A')


Авторизация: 2747
Страница товара: 1012
Корзина: 54
Оплата: 0


In [29]:
user_events_by_steps(events_groups[events_groups['group'] == 'A'], 'Конверсия пользователей (последовательно) в группе В')

Авторизация: 2747
Страница товара: 1012
Корзина: 54
Оплата: 0


Количество пользователей, которые последователно проходили все этапы сайта последовательно, не очень большое - следовательно,  на страницу оплаты можно попасть другими способами - с других страниц сайта или стронних ресурсов (из директа, например).    
Из группы А шаг за шагом до оплаты не дошел ни один пользователь. Из группы В только один. В анализе мы будем рассматривать всех пользователей, вне зависиости от их пути на сайте.

### Анализ распределения количества событий на пользователя в выборках<a id="distribution"></a>

In [30]:
# Проверим сколько уникальных тестов было проведено
final_ab_participants['ab_test'].unique()

array(['recommender_system_test', 'interface_eu_test'], dtype=object)

Было проведено 2 теста recommender_system_test и interface_eu_test

In [31]:
# Создадим датасет с тестом рекомендательной системы (recommender_system_test)
recommender_system_test = final_ab_participants[final_ab_participants['ab_test'] == 'recommender_system_test'].copy()
#display(recommender_system_test.head())

In [32]:
# Разделим recommender_system_test на группы А и В
group_a = recommender_system_test[recommender_system_test['group'] == 'A'].drop('ab_test', axis=1)
group_b = recommender_system_test[recommender_system_test['group'] == 'B'].drop('ab_test', axis=1)

In [33]:
# Подсчитаем чило событий для каждого пользователя
user_events_count = final_ab_events.groupby('user_id').agg({'event_name':'count'})
#display(user_events_count.head())

In [34]:
# Объеним группы теста и число событий каждого пользователя
group_a_user_events = user_events_count.merge(group_a, how='inner', on='user_id')
group_b_user_events = user_events_count.merge(group_b, how='inner', on='user_id')
# Объеденим группы в один датасет
group_user_events = pd.concat([group_a_user_events, group_b_user_events])

In [35]:
# Постороим гистограмму распеределения событий для обеих групп
px.histogram(group_user_events, x='event_name', color='group', title='Распеределение событий для обеих групп')

In [36]:
print('Среднее количество событий в группе А -', round(group_a_user_events['event_name'].mean(), 2))
print('Среднее количество событий в группе B -',  round(group_b_user_events['event_name'].mean(), 2))

Среднее количество событий в группе А - 7.03
Среднее количество событий в группе B - 5.81


Число событий на одного пользователя выше в группе А.


### Присутствие в выборках одни и те же пользователи, пересечение с аудиториями других тестов<a id="double"></a>

In [37]:
double = group_a_user_events.merge(group_b_user_events, how='inner', on = 'user_id')
display(double)

,user_id,event_name_x,group_x,event_name_y,group_y


Пользователей попавших в обе группы - нет.   
**Посмотрим есть ли пользователи попавших в оба теста - recommender_system_test и interface_eu_test**

In [38]:
# Создадим датасет с тестом interface_eu_test
interface_eu_test = final_ab_participants[final_ab_participants['ab_test'] == 'interface_eu_test'].copy()
cross_users = interface_eu_test.merge(recommender_system_test, how='inner', on = 'user_id')['user_id']
print('Число пользователей участвовавших в обоих тестах - ', cross_users.count())

Число пользователей участвовавших в обоих тестах -  1602


### Распределение событий по дням и пересчение с другими маркетинговыми активностями<a id="cross"></a>


In [39]:
# Сосчитаем число событий по дням
final_ab_events['date'] = final_ab_events['event_dt'].dt.date
user_events = final_ab_events.groupby('date').agg({'event_name':'count'}).reset_index()
display(user_events.head())

,date,event_name
0,2020-12-07,11385
1,2020-12-08,12547
2,2020-12-09,12122
3,2020-12-10,14077
4,2020-12-11,13864


In [40]:
# Визуализируем 
px.bar(user_events, x='date', y='event_name', title='Число событий по дням', text='event_name')

In [41]:
# Сосчитаем по группам
events_groups['date'] = events_groups['event_dt'].dt.date
user_events_a = events_groups[events_groups['group'] == 'A'].groupby('date').agg({'event_name':'count'}).reset_index()
user_events_b = events_groups[events_groups['group'] == 'B'].groupby('date').agg({'event_name':'count'}).reset_index()

In [42]:
# Визуализируем А
px.bar(user_events_a, x='date', y='event_name', title='Число событий по дням - Группа А', text='event_name')

In [43]:
# Визуализируем В
px.bar(user_events_b, x='date', y='event_name', title='Число событий по дням - Группа B', text='event_name')

В обеих группах пик числа пользователей пришёлся на 21 декабря.

In [44]:
# Проверим были пересекался ли тест с другми маркетинговыми активностями
cross_events = ab_project_marketing_events[(ab_project_marketing_events['start_dt'] > '2020-12-07') &
                            (ab_project_marketing_events['start_dt'] < '2021-01-04')]
display(cross_events)

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
10,CIS New Year Gift Lottery,CIS,2020-12-30,2021-01-07


Резкий рост событий в группе А 14 декабря может быть связан с запуском маркетинговой компании Christmas&New Year Promo и покупкой подарков перед праздниками - это также негативно влияет на корректоность провдения теста.

### Равномерность распределения по тестовым группам и правильность их формирования<a id="equability"></a>

**Проверим была ли аудитория теста сформирована правильно**

In [45]:
# Подсчистаем общее количество новых пользователей региона EU
EU_new = final_ab_new_users[final_ab_new_users['region'] == 'EU']['user_id'].nunique()
print('Общее количество новых пользователей из региона EU - ', EU_new)

Общее количество новых пользователей из региона EU -  46270


In [46]:
# Расчитаем долю новых пользователей региона EU попавших в тест
EU_test = final_ab_new_users.merge(final_ab_participants[final_ab_participants['ab_test'] == 'recommender_system_test'],
                                    how="inner",
                                    on='user_id')
print('Процент аудитории из EU', round(EU_test['user_id'].nunique()/(EU_new/100), 2), '%')

Процент аудитории из EU 14.48 %


В условиях теста указано, что 15% новой аудитории из EU, но в тест попало 14.48%

**Посмотрим было ли распределение по тестовым группам равномерным**

In [47]:
rs_test_a = final_ab_participants[(final_ab_participants['ab_test'] == 'recommender_system_test') &
                                                  (final_ab_participants['group'] == 'A')]
rs_test_b = final_ab_participants[(final_ab_participants['ab_test'] == 'recommender_system_test') &
                                                  (final_ab_participants['group'] == 'B')]
print('Количество пользователей в группе А -', rs_test_a['user_id'].nunique())
print('Количество пользователей в группе B -', rs_test_b['user_id'].nunique())
print('Разница в пользу группы А -', rs_test_a['user_id'].nunique()-rs_test_b['user_id'].nunique())

Количество пользователей в группе А - 3824
Количество пользователей в группе B - 2877
Разница в пользу группы А - 947


Пользователи распеределены по группам неравномерно

**Выводы**   
* В обеих группах больше половины пользователей, зарегистрировавшихся на сайте, перешли на страницу с товаром. Почти половина пользователей со страницы с товаром перешли к оплате товаров.     
* Количество оплаченных товаров превышает количество товаров, добавленных в корзину - скорей всего у пользователей есть возможность перейти к оплате товара сразу со страницы товара.     
* Количество пользователей, которые последовательно проходили все этапы сайта последовательно, не очень большое - следовательно, на страницу оплаты можно попасть другими способами - с других страниц сайта или стронных ресурсов (из директа, например).    
* Из группы А шаг за шагом до оплаты не дошел ни один пользователь. Из группы В только один.
* Число событий на одного пользователя выше в группе А.    
* Мы установили, что среди пользователей есть пересечение с аудиторией теста interface_eu_test, и по времени есть пересечение с другими маркетинговыми активностями.   
* В обеих группах пик числа пользователей пришёлся на 21 декабря.     
* В условиях теста указано, что 15% новой аудитории из EU, но в тест попало 14.48%.   
* Пользователи распределены по группам неравномерно – в группе А их больше.    
* Резкий рост событий в группе А 14 декабря может быть связан с запуском маркетинговой компании Christmas&New Year Promo и покупкой подарков перед праздниками - это также негативно влияет на корректоность провдения теста, и мы учтём это при формулировании выводов.     
!!! АВ-тест выпал на время новогодней промо-акции, что делают его валидность сомнительной, новый год итак время для покупок и не показательно для аб теста, здесь еще и маркетинговая активность.


---

## Оценка результатов А/В-теста<a id="AB_test"></a>   
Мы проанализируем:   
* Результаты A/В-тестирования   
* Проверим статистическую разницу долей z-критерием

In [48]:
# Выделем нужные нагруппы пользователей 
# Объедним данные групп  с данными пользователей, зарегистрировавшиеся с 7 по 21 декабря 2020 года
group_a_total = group_a.merge(final_ab_new_users, how = 'inner', on='user_id')
group_b_total = group_b.merge(final_ab_new_users, how = 'inner', on='user_id')

# Выделим пользователей из  региона EU
group_a_EU = group_a_total[group_a_total['region'] == 'EU'].copy()
group_b_EU = group_b_total[group_b_total['region'] == 'EU'].copy()

# Оставим 14 дневный период с момента регистрации
group_a_EU  = group_a_EU.merge(final_ab_events, how = 'inner', on = 'user_id')
group_b_EU = group_b_EU.merge(final_ab_events, how = 'inner', on = 'user_id')
group_a_EU = group_a_EU[(group_a_EU['event_dt'] - group_a_EU['first_date']).dt.days < 15]
group_b_EU = group_b_EU[(group_b_EU['event_dt'] - group_b_EU['first_date']).dt.days < 15]

#display(group_a_EU.head())
#display(group_a_EU.head())

### Анализ результата A/B-теста<a id="AB_result"></a>

In [49]:
# Подсчитаем число польователей в группах по событиям
group_a_group = (group_a_EU
                   .groupby('event_name')
                   .agg({'user_id':'nunique'})
                   .sort_values(by = 'user_id', ascending=False)
                   .reset_index())

group_b_group = (group_b_EU
                   .groupby('event_name')
                   .agg({'user_id':'nunique'})
                   .sort_values(by = 'user_id', ascending=False)
                   .reset_index())

In [50]:
# Построим воронку
fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'Группа A',
    y = group_a_group['event_name'],
    x = group_a_group['user_id'],
    textinfo = "value+percent previous"
))

fig.add_trace(go.Funnel(
    name = 'Группа B',
    y = group_b_group['event_name'],
    x = group_b_group['user_id'],
    textinfo = "value+percent previous"
))

fig.update_layout(title_text='Конверсия пользователей в группах А и В')
fig.show()

In [51]:
# Объеденим группы с действия новых пользователей в период с 7 декабря 2020 по 4 января
group_a_all = group_a_EU.merge(final_ab_events, how = 'outer', on = 'user_id')
group_b_all = group_b_EU.merge(final_ab_events, how = 'outer', on = 'user_id')

# Выделим группы А и В
group_a_all = group_a_all[group_a_all['group'] == 'A']
group_b_all = group_b_all[group_b_all['group'] == 'B']

group_a_all['event_name'] = group_a_all['event_name_x'].fillna('no_event')
group_b_all['event_name'] = group_b_all['event_name_x'].fillna('no_event')

#display(group_a_all.head())
#display(group_b_all.head())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [52]:
# Подсчитаем число польователей в группах по событиям
group_a_group_all = (group_a_all
                   .groupby('event_name')
                   .agg({'user_id':'nunique'})
                   .sort_values(by = 'user_id', ascending=False)
                   .reset_index())
group_b_group_all = (group_b_all
                   .groupby('event_name')
                   .agg({'user_id':'nunique'})
                   .sort_values(by = 'user_id', ascending=False)
                   .reset_index())

In [53]:
fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'Группа A',
    y = group_a_group_all['event_name'],
    x = group_a_group_all['user_id'],
    textinfo = "value+percent previous"
))

fig.add_trace(go.Funnel(
    name = 'Группа B',
    y = group_b_group_all['event_name'],
    x = group_b_group_all['user_id'],
    textinfo = "value+percent previous"
))

fig.update_layout(title_text='Конверсия пользователей в группах А и В')
fig.show()

**Конверсия пользователей в группах А и В соспоставимы**

### Проверка статистической разницы долей z-критерием<a id="AB_z"></a>      
   
H0: Между долями нет статистической значимой разницы  - доли равны   
H1: Между долями есть статистически значимая разница - доли не равны   

In [54]:
# Зададим уровень статистической значимости равный 5%
alpha = 0.05

In [55]:
# Зададим функция проверки Z - критерием
def z_test(successes, trials, alpha):
    # пропорция успехов в первой группе:
    p1 = successes[0]/trials[0]
    
    # пропорция успехов во второй группе:
    p2 = successes[1]/trials[1]

    # пропорция успехов в комбинированном датасете:
    p_combined = (successes[0] + successes[1]) / (trials[0] + trials[1])

    # разница пропорций в датасетах
    difference = p1 - p2

    # считаем статистику в ст.отклонениях стандартного нормального распределения
    z_value = abs(difference / mth.sqrt(p_combined * (1 - p_combined) * (1/trials[0] + 1/trials[1])))

    # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
    distr = st.norm(0, 1)
    p_value = (1 - distr.cdf(abs(z_value))) * 2

    print('p-значение: ', p_value)

    if (p_value < alpha):
        print("Отвергаем нулевую гипотезу")
    else:
        print("Не получилось отвергнуть нулевую гипотезу")

In [56]:
# Объеденим группы в один датасет и сгруппируем по событиям
group = (group_a_group_all[group_a_group_all['event_name'] != 'no_event']
                 .merge(group_b_group_all, how='left', on='event_name')
                 .rename(columns={'user_id_x':'Группа А',
                                  'user_id_y':'Группа В'}))
group['total'] = group.sum(axis=1)
display(group.head())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



,event_name,Группа А,Группа В,total
0,login,2604,876,3480
1,product_page,1685,493,2178
2,purchase,833,249,1082
3,product_cart,782,244,1026


In [57]:
# Для события login проверка статистической разницы долей z-критерием невозможна, так как это действие совершили все пользователи группы А и группы В
event = ['product_page', 'purchase', 'product_cart']

for index, event in zip(group.loc[1:].index.tolist(), event):
    try:
        print('\033[1m' + event + '\033[0m')
        successes = [group.loc[index][1], group.loc[index][2]]
        trials = [2604, 876]
        z_test(successes, trials, alpha)
        print('\n')
    except KeyError:
        break

product_page
p-значение:  8.195976000324734e-06
Отвергаем нулевую гипотезу


purchase
p-значение:  0.04864766695042433
Отвергаем нулевую гипотезу


product_cart
p-значение:  0.2215941567364419
Не получилось отвергнуть нулевую гипотезу




**Выводы:**  
1. Для события login проверка статистической разницы долей z-критерием невозможна, так как это действие совершили все пользователи групп А и  В, а это число соответствует максимальному размеру выборки групп.   
   
1. Различие межд группами А и В для событий product_page и purchase статистически значимо (p-value < 0.05).
   
1. Различие между экспериментальной группой В и контрольной А для события на этапе product_cart статистически значимо. Но, высокое p-value (выше критерия значимости 0.05) может означать, что мы имеем высокую вероятность случайных различий между показателями, обусловленных обычной дисперсий/вариацией в данных, а не реальной разницей в поведении пользователей.

---

## Выводы и рекомендации<a id="summary"></a>

### Выводы:
1. Мы загрузили и исследовали данные, обнаруженные пропуски данных и несоответствие типов данных колонок их содержимому - исправили.   
1. Исследовательский анализ данных показал:    
* В обеих группах больше половины пользователей, зарегистрировавшихся на сайте, перешли на страницу с товаром. Почти половина пользователей со страницы с товаром перешли к оплате товаров.     
* Количество оплаченных товаров превышает количество товаров, добавленных в корзину - скорей всего у пользователей есть возможность перейти к оплате товара сразу со страницы товара.     
* В воронке получилось 4 варианта событий:   
>login - регистрация пользователя   
product_page - посещение страницы с товаром   
product_cart - добавление товара в корзину   
 purchase – заказ   
* Количество пользователей, которые последовательно проходили все этапы сайта последовательно, не очень большое - следовательно, на страницу оплаты можно попасть другими способами - с других страниц сайта или стронных ресурсов (из директа, например).    
* Из группы А шаг за шагом до оплаты не дошел ни один пользователь. Из группы В только один.
* Число событий на одного пользователя выше в группе А.    
* Мы установили, что среди пользователей есть пересечение с аудиторией теста interface_eu_test, и по времени есть пересечение с другими маркетинговыми активностями.   
* В обеих группах пик числа пользователей пришёлся на 21 декабря.    
3. Анализ АВ- теста показал:   
* В условиях теста указано, что 15% новой аудитории из EU, но в тест попало 14.48%.   
* Пользователи распределены по группам неравномерно – в группе А их больше.   
*  АВ-тест выпал на время новогодней промо-акции, что делают его валидность сомнительной, новый год итак время для покупок и не показательно для аб теста, здесь еще и маркетинговая активность.  
* Для события login проверка статистической разницы долей z-критерием невозможна, так как это действие совершили все пользователи групп А и  В, а это число соответствует максимальному.   
* Различие между группами А и В для событий product_page и purchase статистически значимо (p-value < 0.05).   
* Различие между экспериментальной группой В и контрольной А для события на этапе product_cart статистически значимо. Но, высокое p-value (выше критерия значимости 0.05) может означать, что мы имеем высокую вероятность случайных различий между показателями, обусловленных обычной дисперсий/вариацией в данных, а не реальной разницей в поведении пользователей. 
    
**В итоге нельзя говорить об успешности теста -  есть ряд ошибок при его проведении.** 
   
### Рекомендация
НЕ раскатывать новую систему рекомендаций на всех пользователей, а скорректировать настройки A/B теста и запустить его повторно.    
